In [ ]:
import os
import argparse
import warnings
import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm
from python_speech_features import mfcc

# определим функцию распознавания аргументов (заполнить позже)
def build_arg_parser():
    parser = argparse.ArgumentParser(description='Trains \ the HMM-based speech recognition system')
    parser.add_argument("--input-folder")

#Определяем класс для тренировки HMM
class ModelHMM(object):
    def __init__(self, num_components=4, num_iter = 1000):
        self.n_components = num_components
        self.n_iter = num_iter
        self.cov_type = "diag" # тип ковариации
        self.model_name = "GaussianHMM"# тип марковской модели
        self.models = [] # модели для каждого слова
        #определение модели по указанным параметрам
        self.model = hmm.GaussianHMM(n_components=self.n_components,
                                      covariance_type=self.cov_type,
                                      n_iter= self.n_iter)

    #train_data -- двумерный массив по 13 элементов в каждой строке
    # определим метод для обучения модели
    def train (self, train_data):
        np.seterr(all='ignore')
        cur_model = self.model.fit(train_data)
        self.models.append(cur_model)

    # создадим метод для оценки входных данных
    def compute_score(self, input_data):
        return self.model.score(input_data)


# определим функцию для построения модели для каждого слова в тренировочном наборе данных
def build_models (input_folder):
    speech_models = []
    for dirname in os.listdir(input_folder):
        subfolder = os.path.join(input_folder, dirname)
        if not os.path.isdir(subfolder):
            continue
        label = subfolder[subfolder.rfind('/') +1:]
        X = np.array([])
        #создается список файлов для тренировки модели
        training_files = [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]
        for filename in training_files:
            filepath = os.path.join(subfolder, filename)
            sampling_freq, signal = wavfile.read(filepath)
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                features_mfcc = mfcc(signal, sampling_freq)
            if len(X) == 0:
                X = features_mfcc
            else:
                X = np.append(X, features_mfcc, axis=0)

#инициализируем модель для текущего слова
        model = ModelHMM()
        model.train(X)
        speech_models.append((model, label))
        model = None
    return speech_models





